**ZDRUZEVANJE VSEH DATOTEK**

- uporabim samo tistih 20 stolpcev ki jih res rabim

In [6]:
import pandas as pd
import glob
import os

vsiFili = glob.glob(os.path.join('data/', "*.csv"))

seznamDf = []

print("Začenjam branje datotek...")

for datoteka in vsiFili:
    temp_df = pd.read_csv(datoteka)
    seznamDf.append(temp_df)
    print(f"Datoteka {os.path.basename(datoteka)} je naložena.")

df = pd.concat(seznamDf, ignore_index=True)

display(df.head())
display(df.shape)

print("---")
print(f"Združevanje končano. Skupno število vrstic: {len(df)}")

Začenjam branje datotek...
Datoteka 2025-07-01.csv je naložena.
Datoteka 2025-07-02.csv je naložena.
Datoteka 2025-07-03.csv je naložena.
Datoteka 2025-07-04.csv je naložena.
Datoteka 2025-07-05.csv je naložena.
Datoteka 2025-07-06.csv je naložena.
Datoteka 2025-07-07.csv je naložena.
Datoteka 2025-07-08.csv je naložena.
Datoteka 2025-07-09.csv je naložena.
Datoteka 2025-07-10.csv je naložena.
Datoteka 2025-07-11.csv je naložena.
Datoteka 2025-07-12.csv je naložena.
Datoteka 2025-07-13.csv je naložena.
Datoteka 2025-07-14.csv je naložena.
Datoteka 2025-07-15.csv je naložena.
Datoteka 2025-07-16.csv je naložena.
Datoteka 2025-07-17.csv je naložena.
Datoteka 2025-07-18.csv je naložena.
Datoteka 2025-07-19.csv je naložena.
Datoteka 2025-07-20.csv je naložena.
Datoteka 2025-07-21.csv je naložena.
Datoteka 2025-07-22.csv je naložena.
Datoteka 2025-07-23.csv je naložena.
Datoteka 2025-07-24.csv je naložena.
Datoteka 2025-07-25.csv je naložena.
Datoteka 2025-07-26.csv je naložena.
Datoteka 20

MemoryError: Unable to allocate 41.4 GiB for an array with shape (186, 29844451) and data type float64

**Rough Preprocessing** - isto kot prej

In [ ]:
import pandas as pd

df = pd.read_csv('sample.csv', sep=',')
display(df.head(5))
display(df.shape)

NaN = df.isnull().mean()

nujni_stolpci = [
    'smart_5_raw',
    'smart_187_raw',
    'smart_188_raw',
    'smart_197_raw',
    'smart_198_raw',
    'smart_9_raw',
    'smart_194_raw',
    'capacity_bytes',
    'model',
    'failure'
]

#obdrži stolpec, če ima < 30% NaN ALI če je na seznamu nujnih
relevantni = NaN[(NaN < 0.3) | (NaN.index.isin(nujni_stolpci))].index.tolist()

print(f"Obdržali bomo {len(relevantni)} stolpcev.")

zaStran = [col for col in df.columns if col not in relevantni]
df = df.drop(columns=zaStran)

display(df.shape)
display(df)

# "lokacijski stolpci" od blackblaze fizicnih serverjev, ki jih ne rabimo
lokacijski = df[['datacenter','cluster_id','vault_id','pod_id','pod_slot_num','is_legacy_format']]

#ostali nepomembni
nepomembniOstalo = df[['date','serial_number']]

# vsi "normalized" so samo interpetacija proizvajalca, ker je vec proizvajalcev so nekonsistentni
def vsiNorm(df):
    normStolpci = []

    for col in df.columns:
        if 'normalized' in col:
            normStolpci.append(col)
    return normStolpci

normStolpci = vsiNorm(df)
df = df.drop(columns=normStolpci)
df = df.drop(columns=lokacijski)
df = df.drop(columns=nepomembniOstalo)
display(df.shape)   #19 dejansko uporabnih stolpcev ..
display(df.columns)

#df.to_csv('izbolsani_podatki.csv', index=False)

**Vrste diskov**

In [ ]:
unikatneVrsteDiskov = df['model'].unique()
print("vrste diskov: ", unikatneVrsteDiskov)

ssd_keywords = ['SSD', 'MTFD', 'SSDSC', '850 PRO', '870 EVO', '860 PRO', '5300']
df['jeSSD'] = df['model'].apply(lambda x: 1 if any(k in str(x).upper() for k in ssd_keywords) else 0)

def getModel(model):
    m = str(model).upper()
    if m.startswith('ST') or 'SEAGATE' in m: return 'Seagate'
    if m.startswith('WDC') or m.startswith('WD') or 'WESTERN' in m: return 'Western Digital'
    if m.startswith('HGST') or m.startswith('HUH') or m.startswith('HMS'): return 'HGST'
    if m.startswith('TOSHIBA') or m.startswith('MG'): return 'Toshiba'
    if m.startswith('SAMSUNG'): return 'Samsung'
    if m.startswith('CT') or 'CRUCIAL' in m: return 'Crucial'
    return 'Other'

df['model'] = df['model'].apply(getModel)

display(df)
ostali = df[df['model'] == 'Other']['model'].unique()
print("nekategorizirani modeli: ", ostali)
modeli = df['model'].unique()
display("kategorizirani modeli: ", modeli)

**Temeljit Preprocessing**

POMEMBNO - Zapolni NaN v teh nujnih stolpcih z 0
Ker XGBoost ne mara popolnih praznin, manjše število NaN pa zna rešiti,

In [ ]:
unikatneVrsteDiskov = df['model'].unique()
print("vrste diskov: ", unikatneVrsteDiskov)

ssd_keywords = ['SSD', 'MTFD', 'SSDSC', '850 PRO', '870 EVO', '860 PRO', '5300']
df['jeSSD'] = df['model'].apply(lambda x: 1 if any(k in str(x).upper() for k in ssd_keywords) else 0)

def getModel(model):
    m = str(model).upper()
    if m.startswith('ST') or 'SEAGATE' in m: return 'Seagate'
    if m.startswith('WDC') or m.startswith('WD') or 'WESTERN' in m: return 'Western Digital'
    if m.startswith('HGST') or m.startswith('HUH') or m.startswith('HMS'): return 'HGST'
    if m.startswith('TOSHIBA') or m.startswith('MG'): return 'Toshiba'
    if m.startswith('SAMSUNG'): return 'Samsung'
    if m.startswith('CT') or 'CRUCIAL' in m: return 'Crucial'
    return 'Other'

df['model'] = df['model'].apply(getModel)

display(df)
ostali = df[df['model'] == 'Other']['model'].unique()
print("nekategorizirani modeli: ", ostali)
modeli = df['model'].unique()
display("kategorizirani modeli: ", modeli)

display(df.head())